In [151]:
from skt.gcp import (
    PROJECT_ID,
    bq_insert_overwrite,
    bq_to_df,
    bq_to_pandas,
    get_bigquery_client,
    bq_table_exists,
    get_max_part,
    load_query_result_to_table,
    pandas_to_bq,
    pandas_to_bq_table,
    load_bigquery_ipython_magic,
    get_bigquery_client,
    _print_query_job_results,
    load_query_result_to_partitions
    
)

from skt.ye import (
    get_hdfs_conn,
    get_spark,
    hive_execute,
    hive_to_pandas,
    pandas_to_parquet,
    slack_send,
    get_secrets
)

In [152]:
from skt.vault_utils import get_secrets
proxies = get_secrets('proxies')

In [153]:
template_query = f"""
SELECT * 
FROM adot_reco_dev.profile_template
WHERE source_domain = 'tdeal'
"""

In [154]:
template_df = bq_to_pandas(template_query)

query: 
SELECT * 
FROM adot_reco_dev.profile_template
WHERE source_domain = 'tdeal'

destination: skt-datahub._775c5ccab1096b3cccd7ac34a5db11c0a354fb07.anon34076360561f0729eea34caba080f9829e3bed60bdb11628cbf464588e6ca344
total_rows: 3
slot_secs: 0.023

Downloading: 100%|██████████|


In [155]:
tdeal_long_template = template_df.loc[template_df.property=='long'].template.values[0]
tdeal_short_template = template_df.loc[template_df.property=='short'].template.values[0]
tdeal_state_template = template_df.loc[template_df.property=='state'].template.values[0]

In [156]:
tdeal_state_template

'특징: {state_profile}'

In [157]:
query = f"""
WITH LOW_TABLE AS (
    SELECT  luna_id,
            CASE 
                WHEN longterm_profile='' THEN ""
                ELSE    REGEXP_REPLACE(
                          '{tdeal_long_template}',
                          r'\\{{longterm_profile\\}}',
                          longterm_profile
                    )
            END AS longterm_profile_template,

            CASE 
                WHEN shortterm_profile='' THEN ""
                ELSE 
                    REGEXP_REPLACE(
                    '{tdeal_short_template}',
                    r'\\{{shortterm_profile\\}}',
                        shortterm_profile
                    ) 
            END AS shortterm_profile_template,
            CASE 
                WHEN state_profile='' THEN ""
                ELSE 
                    REGEXP_REPLACE(
                    '{tdeal_state_template}',
                    r'\\{{state_profile\\}}',
                        state_profile
                    ) 
            END AS state_profile_template

    FROM (
        SELECT  luna_id,            
                COALESCE(ARRAY_TO_STRING(long_prefered_domain, ','), '') as longterm_profile,
                COALESCE(ARRAY_TO_STRING(short_prefered_domain, ','), '') as shortterm_profile,
                COALESCE(ARRAY_TO_STRING(user_state, ','), '') as state_profile
        FROM adot_reco_dev.temp_jh_tdeal_profile
        WHERE luna_id is not null
    )
),

MIDDLE_TABLE AS(
    SELECT  luna_id,
            CASE 
                WHEN longterm_profile_template !='' THEN  CONCAT(longterm_profile_template, ',' , shortterm_profile_template)
            ELSE shortterm_profile_template
            END AS tdeal_profile,
            state_profile_template
    FROM LOW_TABLE
)

SELECT  luna_id,
        CASE 
            WHEN state_profile_template !='' THEN  CONCAT(state_profile_template, ',' , tdeal_profile)
        ELSE tdeal_profile
        END AS tdeal_profile
FROM MIDDLE_TABLE
"""

In [162]:
bq_insert_overwrite(sql=query, destination=f'{PROJECT_ID}.adot_reco_dev.adotUser_tdeal_profile_temp')

query: 
WITH LOW_TABLE AS (
    SELECT  luna_id,
            CASE 
                WHEN longterm_profile='' THEN ""
                ELSE    REGEXP_REPLACE(
                          '평소 {longterm_profile} 용품 구매에 관심',
                          r'\{longterm_profile\}',
                          longterm_profile
                    )
            END AS longterm_profile_template,

            CASE 
                WHEN shortterm_profile='' THEN ""
                ELSE 
                    REGEXP_REPLACE(
                    '최근 {shortterm_profile} 용품 구매',
                    r'\{shortterm_profile\}',
                        shortterm_profile
                    ) 
            END AS shortterm_profile_template,
            CASE 
                WHEN state_profile='' THEN ""
                ELSE 
                    REGEXP_REPLACE(
                    '특징: {state_profile}',
                    r'\{state_profile\}',
                        state_profile
                    ) 
            END 

In [158]:
tdeal_profile = bq_to_pandas(query)

query: 
WITH LOW_TABLE AS (
    SELECT  luna_id,
            CASE 
                WHEN longterm_profile='' THEN ""
                ELSE    REGEXP_REPLACE(
                          '평소 {longterm_profile} 용품 구매에 관심',
                          r'\{longterm_profile\}',
                          longterm_profile
                    )
            END AS longterm_profile_template,

            CASE 
                WHEN shortterm_profile='' THEN ""
                ELSE 
                    REGEXP_REPLACE(
                    '최근 {shortterm_profile} 용품 구매',
                    r'\{shortterm_profile\}',
                        shortterm_profile
                    ) 
            END AS shortterm_profile_template,
            CASE 
                WHEN state_profile='' THEN ""
                ELSE 
                    REGEXP_REPLACE(
                    '특징: {state_profile}',
                    r'\{state_profile\}',
                        state_profile
                    ) 
            END 

In [159]:
tdeal_profile.to_csv('template_sample.csv')

In [161]:
tdeal_profile.head(10)

,luna_id,tdeal_profile
0,APL00000D8HTGIIHYZ9C,최근 골프 용품 구매
1,APL00000BYM42M1A248W,최근 골프 용품 구매
2,APL00000CJW5BXEJ2NSW,최근 골프 용품 구매
3,APL00000D4FXYRK04WZK,최근 골프 용품 구매
4,APL00000CASI8AJHAWHS,최근 골프 용품 구매
5,APL00000DGX3K077K6BK,최근 골프 용품 구매
6,APL00000CU3Z4N4K3R40,최근 골프 용품 구매
7,APL00000BR1BB1WK0BUO,최근 골프 용품 구매
8,APL00000D2EJGWFWFWU8,최근 골프 용품 구매
9,APL00000BPJNTODWB1FK,최근 골프 용품 구매
